Configuração Inicial

In [1]:
from dotenv import load_dotenv
import os
import sys
import pandas as pd


In [2]:
# Adicionar caminho absoluto do diretório dos scripts
sys.path.append(os.path.abspath("../scripts"))

from extract_tables import processar_pasta

In [3]:
# Carregar variáveis do arquivo .env
load_dotenv()

openai_key = os.getenv("OPENAI_API_KEY")

In [ ]:
# Configurar caminhos absolutos
caminho_base = os.path.abspath(os.path.join(os.getcwd(), ".."))
pasta_entrada = os.path.abspath(os.path.join(caminho_base, "data", "pdfs"))  # PDFs originais
pasta_saida = os.path.abspath(os.path.join(caminho_base, "data", "processed", "registros"))  # CSVs processados

# Verificar caminhos
print(f"Caminho de entrada: {pasta_entrada}")
print(f"Caminho de saída: {pasta_saida}")

Caminho de entrada: e:\Carla Reis\Projects\Notebooks\fiscal-rag\data\raw
Caminho de saída: e:\Carla Reis\Projects\Notebooks\fiscal-rag\data\processed\registros


1. Extração das regras do Manual do Guia Prático

In [5]:
# Processar todos os PDFs
resultados = processar_pasta(pasta_entrada, pasta_saida)

# Mostrar resultados
print("Resultados do processamento:")
display(resultados)

c:\Users\clr_c\anaconda3\envs\fiscalrag\lib\site-packages\camelot\utils.py:740: UserWarning:   (551.64, 554.13) does not lie in column range (48.71017487395207, 551.2888847138252)
  warnings.warn(
c:\Users\clr_c\anaconda3\envs\fiscalrag\lib\site-packages\camelot\utils.py:740: UserWarning:   (551.64, 554.64) does not lie in column range (48.71017487395207, 551.2888847138252)
  warnings.warn(


Coluna Anterior:Index(['Nº', 'Campo', 'Descrição', 'Tipo', 'Tam', 'Dec', 'Obrig'], dtype='object', name=0)
Coluna Anterior:Index(['Nº', 'Campo', 'Descrição', 'Tipo', 'Tam', 'Dec', 'Obrig'], dtype='object', name=0)
Processado e:\Carla Reis\Projects\Notebooks\fiscal-rag\data\raw\0200-0205.pdf - 2 tabela(s) processada(s).
Coluna Anterior:Index(['Nº', 'Campo', 'Descrição', 'Tipo', 'Tam', 'Dec', 'Obrig'], dtype='object', name=0)
Coluna Anterior:Index(['Nº', 'Campo', 'Descrição', 'Tipo', 'Tam', 'Dec', 'Entr', 'Saída'], dtype='object', name=0)
Coluna Anterior:Index(['Nº', 'Campo', 'Descrição', 'Tipo', 'Tam', 'Dec', 'Entr', 'Saída'], dtype='object', name=0)
Processado e:\Carla Reis\Projects\Notebooks\fiscal-rag\data\raw\C100.pdf - 3 tabela(s) processada(s).
Coluna Anterior:Index(['Nº', 'Campo', 'Descrição', 'Tipo', 'Tam', 'Dec', 'Entr', 'Saída'], dtype='object', name=0)
Coluna Anterior:Index(['Nº', 'Campo', 'Descrição', 'Tipo', 'Tam', 'Dec', 'Entr', 'Saída'], dtype='object', name=0)
Processado

,Arquivo,Status
0,0200-0205.pdf,✅ Sucesso
1,C100.pdf,✅ Sucesso
2,C170.pdf,✅ Sucesso
